# Kryptologie Lab - Übung 05 - Substitutions Permutations Netwerk

In [1]:
from tqdm.notebook import tqdm

In [2]:
sbox = [0xE, 0x4, 0xD, 0x1, 0x2, 0xF, 0xB, 0x8, 0x3, 0xA, 0x6, 0xC, 0x5, 0x9, 0x0, 0x7]
perm = [1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15, 4, 8, 12, 16]

In [3]:
# sbox substitution 
def sbox_block(value, inverse=False):
    if inverse:
        return sbox.index(value)
    else: 
        return sbox[value]

# lokale Substitution durch SBox
# sbox auf jeden der 4 Blocke (mit jeweils 4 Bits)
def executeSbox(value,inverse=False):
    block0 = sbox_block(value & 0xF, inverse)
    block1 = sbox_block((value & 0xF0) >> 4, inverse)
    block2 = sbox_block((value & 0xF00) >> 8, inverse)
    block3 = sbox_block((value & 0xF000) >> 12, inverse)
    return (block3 << 12) | (block2 << 8) | (block1 << 4) | block0

# Globale Permutation der 16 Bit
def permutation(value):
    output = 0
    for i in range(1, 16):
        bit = value & 0x1
        output = output | (bit << (perm[i] - 1))
        value = value >> 1
    return output

def addRoundKey(value, key):
    return value ^ key

In [4]:
# Encrypt via SPN in 4 Runden
# Input: 4 Block á 4 Bit
# Output: 4 Block á 4 Bit
def encrypt(value, key):
    for i in range(3):
        value = addRoundKey(value, key)
        value = executeSbox(value)
        value = permutation(value)
    value = addRoundKey(value, key)
    value = executeSbox(value)
    value = addRoundKey(value, key)
    return value

## Testing

In [5]:
for i in range(0, 16):
    encrypted = sbox_block(i)
    decrypted = sbox_block(encrypted, inverse=True)
    assert i == decrypted

value = 0xABCD
encrypted = executeSbox(value)
decrypted = executeSbox(encrypted, inverse=True)
assert value == decrypted

encrypt(0xABCD, 0xFABE)

50256

## Lineare Kryptonanalyse
Idee: Suche lineare Approximation an S-Boxen

In [6]:
import random

In [7]:
# returns bit of val at position
def getBit(val, index):
    return (val >> (15-index)) & 0x1

# only second and fourth byte and extends to 4 byte
def expandKey(subkey):
    return ((subkey & 0xF) << 8) ^ ((subkey & 0xF0) >> 4)

def getProbabilityOfKeys(pairs):
    possibleKeys = dict()
    # iterate possible keys
    for subkey in range(0xFF+1):
        key = expandKey(subkey)
        hits = 0
        # iterate text-cipher pairs to find correct part of the key
        for pair in pairs:
            x = pair[0]
            y = pair[1]
            v4 = addRoundKey(y, key)
            u4 = executeSbox(v4, inverse=False)
            approx = getBit(x,4) ^ getBit(x,6) ^ getBit(x,7) ^ getBit(u4,5) ^ getBit(u4,7) ^ getBit(u4,13) ^ getBit(u4,15)
            if approx == 0:
                hits += 1
        possibleKeys[subkey] = hits/len(pairs)
    return possibleKeys

def getBestSubKeyOrder(probabilities):
    for data in probabilities.items():
        probabilities.update({data[0]: abs(data[1] - 0.5)})
    return sorted(probabilities.items(), key=lambda item: item[1], reverse=True)

## Generate Text-Cipher Pairs

In [8]:
key = 0xFFAE
amount = 8000

M = []
for i in tqdm(range(amount)):
    input = random.getrandbits(16)
    M.append((input, encrypt(input, key)))

In [9]:
probabilities = getProbabilityOfKeys(M)
subkeysInOrderWithProbability = getBestSubKeyOrder(probabilities)
print(subkeysInOrderWithProbability)
subkeysInOrder = [entry[0] for entry in subkeysInOrderWithProbability]
print(subkeysInOrder)

[(16, 0.02862500000000001), (20, 0.02862500000000001), (27, 0.02862500000000001), (31, 0.02862500000000001), (80, 0.02862500000000001), (84, 0.02862500000000001), (91, 0.02862500000000001), (95, 0.02862500000000001), (160, 0.02862500000000001), (164, 0.02862500000000001), (171, 0.02862500000000001), (175, 0.02862500000000001), (224, 0.02862500000000001), (228, 0.02862500000000001), (235, 0.02862500000000001), (239, 0.02862500000000001), (48, 0.017874999999999974), (52, 0.017874999999999974), (59, 0.017874999999999974), (63, 0.017874999999999974), (112, 0.017874999999999974), (116, 0.017874999999999974), (123, 0.017874999999999974), (127, 0.017874999999999974), (128, 0.017874999999999974), (132, 0.017874999999999974), (139, 0.017874999999999974), (143, 0.017874999999999974), (192, 0.017874999999999974), (196, 0.017874999999999974), (203, 0.017874999999999974), (207, 0.017874999999999974), (17, 0.011875000000000024), (26, 0.011875000000000024), (85, 0.011875000000000024), (94, 0.01187500

## Brutforce in better order

In [10]:
countKeyTries = 0
cracked = False
for key in tqdm(subkeysInOrder):
    countKeyTries += 1
    for brutforceSubkey in range(0xFF+1):
        expandedKey = expandKey(key)
        testKey = ((brutforceSubkey & 0xF) << 4) ^ ((brutforceSubkey & 0xF0) << 8) ^ expandedKey
        cipher = encrypt(M[0][0], testKey)
        if cipher == M[0][1]:
            print("Hit")
            correct = 0
            for m in M:
                cipher = encrypt(m[0], testKey)
                if cipher == m[1]:
                    correct += 1
            if correct == len(M):
                print("Key: ", hex(testKey))
                print("Cracked after ", countKeyTries, " tries")
                cracked = True
                break
    if cracked:
        break       

Hit
Hit
Key:  0xffae
Cracked after  16  tries



With more than 8000 samples the brutforce step becomes even faster. With less, the correct key is more at the back.